In [ ]:
# Import packages
import json
import os
import requests
import datetime
import numpy as np
import pandas as pd
import glob

In [ ]:
# Set target directory
# of logged API json files, will create new one if it does not exist
data_dir = 'rw_api_archive'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
# Download a copy of RW dataset, layer, and widget endpoints. 
# Note 'env' and 'application' params in URL string
api_list = ['dataset','layer','widget']
MAX_ITER = 20

## Handle pagination
def follow_pagination(link_obj):
	if not isinstance(link_obj, dict):
		return (-1, '')
	if link_obj['self'] == link_obj['last']:
		return (0, '')
	return (1, link_obj['next'])

## Loop through each endpoint
for endpoint in api_list:

    data = []
    current_url = f'http://api.resourcewatch.org/v1/{endpoint}?page[size]=1000&env=production&application=rw'

    for i in range(MAX_ITER):
        with requests.get(current_url) as r:
            print(current_url)
            if r.ok:
                ds = json.loads(r.content)
            else:
                raise ValueError(f'API request failed: {current_url}')
        assert 'data' in ds
        assert 'links' in ds

        # Add each page of results together
        data.extend(ds['data'])

        # Stop if there are no more results
        code, link = follow_pagination(ds['links'])
        if code == 1:
            current_url = link
        elif code == 0:
            print("Last page reached")
            break
        elif code == -1:
            raise TypeError('links object in API response malformed')
        else:
            raise ValueError(f'pagination response malformed or not understood')
        
        
    d8 = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")

    with open(f'{data_dir}/{endpoint}_{d8}.json', 'w') as fp:
            json.dump(data, fp)       


In [ ]:
# select filename of most recent matching json files in target directory and load as json
recent_dataset_dl = max(glob.iglob(f'{data_dir}/datas*.json'), key=os.path.getctime)
recent_layer_dl = max(glob.iglob(f'{data_dir}/layer*.json'), key=os.path.getctime)
recent_widget_dl = max(glob.iglob(f'{data_dir}/widge*.json'), key=os.path.getctime)

with open(recent_dataset_dl) as json_file:
    rw_dataset_data = json.load(json_file)
    
with open(recent_layer_dl) as json_file:
    rw_layer_data = json.load(json_file)
    
with open(recent_widget_dl) as json_file:
    rw_widget_data = json.load(json_file)

In [41]:
# Set substring, will search for string across all json objects
subs = 'globalfishingwatch' #<-- SET SUBSTRING HERE
layers_using = [x for x in rw_layer_data if str(x).count(subs) != 0] 
widgets_using = [x for x in rw_widget_data if str(x).count(subs) != 0] 
datasets_using = [x for x in rw_dataset_data if str(x).count(subs) != 0] 

In [ ]:
# List edit urls of all matching objects
litems = [datasets_using, layers_using, widgets_using]
print(subs)
for i, x in enumerate(['datasets', 'layers', 'widgets']):
    print(x)
    for j in litems[i]:
        print('https://resourcewatch.org/admin/data/{}/{}/edit'.format(x, j['id']))

In [44]:
# Create a list of api endpoints of all matching objects
litems = [datasets_using, layers_using, widgets_using]
results = []
for i, x in enumerate(['dataset', 'layer', 'widget']):
    for j in litems[i]:
        results.append('https://api.resourcewatch.org/v1/{}/{}'.format(x, j['id']))

results


['https://api.resourcewatch.org/v1/dataset/6ccfb266-20cd-4838-82b0-5309987a62f0',
 'https://api.resourcewatch.org/v1/dataset/11f16cb9-def0-4bd5-a60e-50c542b837e3',
 'https://api.resourcewatch.org/v1/layer/30c90ee1-ce3f-4a30-afdb-5929ae91a684',
 'https://api.resourcewatch.org/v1/layer/1f11414a-266c-4ca0-83ae-6f07ab7e85df',
 'https://api.resourcewatch.org/v1/layer/02066566-f854-46b7-816a-6810a715e3c0',
 'https://api.resourcewatch.org/v1/layer/23f9c85a-c2bf-4589-a6b9-5e8e434e5474',
 'https://api.resourcewatch.org/v1/layer/359dc32b-8724-456e-bbe7-bfbc19b4b789',
 'https://api.resourcewatch.org/v1/layer/516c8b3f-47e3-486f-a6c6-7c31ff226d98',
 'https://api.resourcewatch.org/v1/layer/87c66011-0859-4042-9b71-94d2376f7b35',
 'https://api.resourcewatch.org/v1/layer/68ee1a1d-770d-4efb-be4e-eaace0d97bb0',
 'https://api.resourcewatch.org/v1/layer/1ecc70ac-bf72-4d2a-9a84-196cf86be0d5',
 'https://api.resourcewatch.org/v1/layer/3a9b0873-4abf-4ad8-8845-0201bd7b88ee',
 'https://api.resourcewatch.org/v1/l